In [0]:
silver_layer_df = spark.table('az_adb_simbus_training.imdb_project_schema.silver_popular_actor_directors')


#display(silver_layer_df)

In [0]:
from pyspark.sql.functions import col, avg, sum as sum_,round,count
from pyspark.sql import functions as F

gold_sum_movies_df = silver_layer_df.groupBy("person_name")\
    .agg(
        count("movie_title").alias("movie_count")
    )

#filtering out the person who has a count more than 10 movies
filtered_gold_sum_movies_df = gold_sum_movies_df.filter(col("movie_count") > 10)
silver_layer_df = silver_layer_df.join(filtered_gold_sum_movies_df,silver_layer_df.person_name == filtered_gold_sum_movies_df.person_name, how="inner")

silver_layer_df=silver_layer_df.drop(filtered_gold_sum_movies_df["person_name"])
#display(silver_layer_df)



In [0]:

gold_avgrate_df = silver_layer_df.groupBy(silver_layer_df['person_name'])\
    .agg(
        round(avg("average_rating"), 2).alias("average_rating"),
        F.count("*").alias("movie_count"),
        F.first("role").alias("role")
       )

#display(gold_avgrate_df)

In [0]:
gold_totalvotes_df = silver_layer_df.groupBy("person_name")\
    .agg(
        sum_("num_votes").alias("total_votes")
       )

#display(gold_totalvotes_df)

In [0]:
final_gold_layer_table=gold_totalvotes_df.join(gold_avgrate_df,gold_totalvotes_df.person_name == gold_avgrate_df.person_name, how="inner").drop(gold_avgrate_df["person_name"])

display(final_gold_layer_table)

final_gold_layer_table.write.format("delta").mode("overwrite").saveAsTable("az_adb_simbus_training.imdb_project_schema.golden_popular_actors_directors")
